### **Paso 1: Configurar Google Colab para Streamlit**

In [1]:
!pip install -qU streamlit openai pyngrok


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 23.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 327.4/327.4 kB 38.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 28.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 65.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 9.6 MB/s eta 0:00:00


### **Paso 2: Código para cada aplicación de chat**

In [27]:
import streamlit as st
import openai
from openai import OpenAI
import random
import time
import os
import subprocess
from pyngrok import ngrok

# 1. Echo Bot

In [3]:
st.title("Echo Bot")

2024-06-22 09:35:31.899 
  command:

    streamlit run /usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py [ARGUMENTS]


DeltaGenerator()

In [4]:
# Inicializar el historial de chat
if "messages" not in st.session_state:
    st.session_state.messages = []

2024-06-22 09:35:44.355 Session state does not function when running a script without `streamlit run`


In [5]:
# Mostrar mensajes del historial en la recarga de la app
for message in st.session_state.messages:
    with st.chat_message(message["role"]):
        st.markdown(message["content"])

In [6]:
# Reaccionar a la entrada del usuario
if prompt := st.chat_input("What is up?"):
    # Mostrar el mensaje del usuario en el contenedor de mensajes del chat
    st.chat_message("user").markdown(prompt)
    # Añadir el mensaje del usuario al historial de chat
    st.session_state.messages.append({"role": "user", "content": prompt})

    response = f"Echo: {prompt}"
    # Mostrar la respuesta del asistente en el contenedor de mensajes del chat
    with st.chat_message("assistant"):
        st.markdown(response)
    # Añadir la respuesta del asistente al historial de chat
    st.session_state.messages.append({"role": "assistant", "content": response})

# 2. Simple Chatbot GUI con streaming

In [7]:
# Emulador de respuesta en streaming
def response_generator():
    response = random.choice(
        [
            "Hello there! How can I assist you today?",
            "Hi, human! Is there anything I can help you with?",
            "Do you need help?",
        ]
    )
    for word in response.split():
        yield word + " "
        time.sleep(0.05)

In [8]:
st.title("Simple chat")

DeltaGenerator()

In [9]:
# Inicializar el historial de chat
if "messages" not in st.session_state:
    st.session_state.messages = []

In [10]:
# Mostrar mensajes del historial en la recarga de la app
for message in st.session_state.messages:
    with st.chat_message(message["role"]):
        st.markdown(message["content"])

In [11]:
# Aceptar la entrada del usuario
if prompt := st.chat_input("What is up?"):
    # Añadir el mensaje del usuario al historial de chat
    st.session_state.messages.append({"role": "user", "content": prompt})
    # Mostrar el mensaje del usuario en el contenedor de mensajes del chat
    with st.chat_message("user"):
        st.markdown(prompt)

    # Mostrar la respuesta del asistente en el contenedor de mensajes del chat
    with st.chat_message("assistant"):
        response = st.write_stream(response_generator())
    # Añadir la respuesta del asistente al historial de chat
    st.session_state.messages.append({"role": "assistant", "content": response})

# 3. Aplicación similar a ChatGPT

In [12]:
st.title("ChatGPT-like clone")

DeltaGenerator()

In [19]:
# Configuramos la API key de OpenAI
openai_api_key = "YOUR_API_KEY"
os.environ["OPENAI_API_KEY"] = openai_api_key
openai.api_key = os.getenv("OPENAI_API_KEY")

In [23]:
if "openai_model" not in st.session_state:
    st.session_state["openai_model"] = "gpt-3.5-turbo"

if "messages" not in st.session_state:
    st.session_state.messages = []

In [24]:
# Mostrar mensajes del historial en la recarga de la app
for message in st.session_state.messages:
    with st.chat_message(message["role"]):
        st.markdown(message["content"])

In [25]:
# Aceptar la entrada del usuario
if prompt := st.chat_input("What is up?"):
    st.session_state.messages.append({"role": "user", "content": prompt})
    with st.chat_message("user"):
        st.markdown(prompt)

    with st.chat_message("assistant"):
        # Llamada a la API de OpenAI para generar una respuesta
        response = openai.ChatCompletion.create(
            model=st.session_state["openai_model"],
            messages=[
                {"role": m["role"], "content": m["content"]}
                for m in st.session_state.messages
            ],
            stream=True,
        )
        # Mostrar la respuesta del asistente en el contenedor de mensajes del chat
        response_content = "".join([chunk['choices'][0]['delta'].get('content', '') for chunk in response])
        st.markdown(response_content)

    # Añadir la respuesta del asistente al historial de chat
    st.session_state.messages.append({"role": "assistant", "content": response_content})

### **Paso 3: Ejecutar Streamlit en Google Colab**

In [30]:
# Especificar tu authtoken de ngrok aquí
ngrok_authtoken = "YOUR_API_KEY"

In [31]:
# Autenticar ngrok con tu authtoken
ngrok.set_auth_token(ngrok_authtoken)

In [32]:
# Crear un archivo para la aplicación Streamlit (por ejemplo, app.py)
with open('app.py', 'w') as f:
    f.write("""
import streamlit as st

st.title("Echo Bot")

# Initialize chat history
if "messages" not in st.session_state:
    st.session_state.messages = []

# Display chat messages from history on app rerun
for message in st.session_state.messages:
    with st.chat_message(message["role"]):
        st.markdown(message["content"])

# React to user input
if prompt := st.chat_input("What is up?"):
    # Display user message in chat message container
    st.chat_message("user").markdown(prompt)
    # Add user message to chat history
    st.session_state.messages.append({"role": "user", "content": prompt})

    response = f"Echo: {prompt}"
    # Display assistant response in chat message container
    with st.chat_message("assistant"):
        st.markdown(response)
    # Add assistant response to chat history
    st.session_state.messages.append({"role": "assistant", "content": response})
""")

In [33]:
# Ejecutar la aplicación Streamlit en segundo plano
process = subprocess.Popen(['streamlit', 'run', 'app.py'])

In [36]:
# Crear un túnel a través de ngrok
public_url = ngrok.connect(8501, "http")
print(f'Public URL: {public_url}')

Public URL: NgrokTunnel: "https://3473-34-16-179-107.ngrok-free.app" -> "http://localhost:8501"


In [37]:
# Mantener el proceso de Streamlit en ejecución
process.wait()

KeyboardInterrupt: 